# 🎯 YOLO Training with Custom Data Augmentation

This notebook trains a YOLO model for malaria detection with **custom filter-based data augmentation**.

**Features:**
- Interactive configuration with widgets
- Custom data augmentation pipeline using image filters
- Support for multiple YOLO versions (v3, v5, v8, etc.)
- Automated training and testing workflow

> 📚 For more information: [filter_learning_framework](https://github.com/andreolli-davide/filter_learning_framework)

## 1. Setup
Install required dependencies.

In [ ]:
%pip install ultralytics

## 2. Configuration
Configure training options using interactive widgets.

In [ ]:
from ipywidgets import widgets

data_augmentation = widgets.Checkbox(
    value=True, description="Apply data augmentation", disabled=False
)

display(data_augmentation)

## 3. Load Dataset
Download and load the malaria detection dataset.

In [ ]:
from pathlib import Path
import kagglehub
from src.dataset import Dataset

DATASET_PATH: Path = Path("resources/dataset")

if not DATASET_PATH or not DATASET_PATH.exists():
    dataset_path = Path(
        kagglehub.dataset_download(
            "davidesenette/malaria-hcm-lcm-1000",
        )
    )
dataset = Dataset.load_from_directory(DATASET_PATH)

## 4. Prepare YOLO Configuration
Generate YAML configuration file for YOLO training.

In [ ]:
from src.yolo import YoloConfig

# Create YOLO configuration pointing to HCM (source) images
config = YoloConfig(path=dataset.base_path / "source")
config_path = config.store_yaml()

print(f"YOLO config saved to: {config_path}")

## 5. Custom Data Augmentation
Implement custom filter-based data augmentation for YOLO training.

### 5.1 Random Hyperparameter Generator
Function to randomly select filter hyperparameters within valid ranges.

### 5.2 Custom YOLO Dataset with Filter Augmentation
Custom dataset class that applies random filter combinations during training.

In [ ]:
from src.filters import FilterParametersHint
from ultralytics.data import YOLODataset
import random


def choose_random_hyperparameters(filters_list):
    """
    Generate random hyperparameters for a list of filters.

    Args:
        filters_list: List of filter adapter classes

    Returns:
        List of filter parameter instances with randomized values
    """
    hyperparameters = []

    for filter_class in filters_list:
        filter_parameters_class = filter_class.parameters_class
        filter_suggested_parameters = {}

        # Iterate over each hyperparameter field in the filter's parameter class
        for field_name, field_info in filter_parameters_class.model_fields.items():
            field_type = field_info.annotation
            type_hints = filter_parameters_class.__annotations__[field_name]

            # Extract hyperparameter hints from type metadata
            for field_hint in type_hints.__metadata__:
                if isinstance(field_hint, FilterParametersHint):
                    # Generate random integer hyperparameters
                    if field_type is int:
                        step = field_hint.step if field_hint.step is not None else 1
                        values = range(
                            field_hint.lower_bound, field_hint.upper_bound + 1, step
                        )
                        filter_suggested_parameters[field_name] = random.choice(
                            list(values)
                        )

                    # Generate random float hyperparameters
                    elif field_type is float:
                        if field_hint.step is not None:
                            # Discrete float (with step)
                            num_steps = (
                                int(
                                    (field_hint.upper_bound - field_hint.lower_bound)
                                    / field_hint.step
                                )
                                + 1
                            )
                            values = [
                                field_hint.lower_bound + i * field_hint.step
                                for i in range(num_steps)
                            ]
                            filter_suggested_parameters[field_name] = random.choice(
                                values
                            )
                        else:
                            # Continuous float (without step)
                            filter_suggested_parameters[field_name] = random.uniform(
                                field_hint.lower_bound, field_hint.upper_bound
                            )

        # Store the suggested parameters for this filter
        hyperparameters.append(
            filter_class.parameters_class(**filter_suggested_parameters)
        )

    return hyperparameters

In [ ]:
import torch
from src.filters import (
    NoOpFilterAdapter,
    MedianBlurFilterAdapter,
    BilateralFilterAdapter,
    SaturationBoostFilterAdapter,
    ClaheFilterAdapter,
    GammaCorrectionFilterAdapter,
    UnsharpMaskFilterAdapter,
    LaplacianSharpenFilterAdapter,
)


class YOLODatasetCustomDA(YOLODataset):
    """
    Custom YOLO dataset with filter-based data augmentation.

    Applies random filter combinations during training to augment images.
    Filter layers:
    1. Noise Reduction: NoOp, MedianBlur, BilateralFilter
    2. Color Correction: NoOp, SaturationBoost
    3. Contrast Enhancement: NoOp, CLAHE, GammaCorrection
    4. Edge Sharpening: NoOp, UnsharpMask, LaplacianSharpen
    """

    def __getitem__(self, index):
        # Get sample from parent dataset
        sample = super().__getitem__(index)

        # Get the image as a PyTorch tensor (CHW format)
        img_tensor = sample["img"]

        # Convert to NumPy HWC format for filter application
        img_numpy = img_tensor.permute(1, 2, 0).cpu().numpy()

        # Define filter layers (one filter randomly selected per layer)
        filter_layers = [
            [NoOpFilterAdapter, MedianBlurFilterAdapter, BilateralFilterAdapter],
            [NoOpFilterAdapter, SaturationBoostFilterAdapter],
            [NoOpFilterAdapter, ClaheFilterAdapter, GammaCorrectionFilterAdapter],
            [
                NoOpFilterAdapter,
                UnsharpMaskFilterAdapter,
                LaplacianSharpenFilterAdapter,
            ],
        ]

        # Randomly select one filter from each layer
        filter_list = [random.choice(layer) for layer in filter_layers]

        # Generate random hyperparameters for selected filters
        hyperparameters = choose_random_hyperparameters(filter_list)

        # Apply filters sequentially
        for i, filter_class in enumerate(filter_list):
            img_numpy = filter_class.apply_filter(img_numpy, hyperparameters[i])

        # Convert back to PyTorch tensor (CHW format) and restore device
        sample["img"] = (
            torch.from_numpy(img_numpy).permute(2, 0, 1).to(img_tensor.device)
        )

        return sample

## 6. YOLO Trainer Class
Wrapper class for training and testing YOLO models.

In [ ]:
from ultralytics import YOLO


class YOLOTrainer:
    """
    YOLO training manager supporting multiple versions (v3, v5, v8, etc.).

    Provides methods for:
    - Loading pretrained models
    - Training with custom parameters
    - Testing and evaluation
    """

    def __init__(self, model_size: str = "s", model_version: str = "v8"):
        """
        Initialize YOLO trainer.

        Args:
            model_size: Model size ('n', 's', 'm', 'l', 'x')
                       For YOLOv3: 'n'=tiny, 'u'=standard, 'l'/'x'=spp
            model_version: Model version ('v3', 'v5', 'v8', etc.)
        """
        self.model_size = model_size
        self.model_version = model_version
        self.model = None

    def load_pretrained(self):
        """Load pretrained COCO weights."""
        if self.model_version == "v3":
            # YOLOv3 Ultralytics: yolov3u.pt, yolov3-tinyu.pt, yolov3-sppu.pt
            size_map = {"tiny": "-tiny", "spp": "-spp", "u": ""}
            suffix = size_map.get(self.model_size, "")
            model_name = f"yolov3{suffix}u.pt"
        else:
            # For v5, v8, v9, v10, v11
            model_name = f"yolo{self.model_version}{self.model_size}.pt"

        self.model = YOLO(model_name)
        print(f"✓ Pretrained model loaded: {model_name}")

    def train(
        self,
        data_yaml: str,
        epochs=100,
        img_size=640,
        batch_size=16,
        lr0=0.01,
        optimizer="SGD",
        device=0,
        patience=100,
        name="exp",
        resume=False,
        momentum=0.9,
        **kwargs,
    ):
        """
        Train YOLO model with paper-recommended parameters.

        Default settings follow YOLOv3 paper (Section 6.1):
        - epochs: 100
        - lr0: 0.01 (initial learning rate)
        - optimizer: SGD
        - momentum: 0.9

        Args:
            data_yaml: Path to YAML configuration file
            epochs: Number of training epochs
            img_size: Input image size
            batch_size: Batch size
            lr0: Initial learning rate
            optimizer: Optimizer type ('SGD', 'Adam', etc.)
            device: GPU device ID or 'cpu'
            patience: Early stopping patience
            name: Experiment name
            resume: Resume from checkpoint
            momentum: SGD momentum
            **kwargs: Additional training arguments

        Returns:
            dict: Training results with best model path
        """
        self.load_pretrained()

        # Training parameters (paper-based defaults + no default augmentation)
        train_params = {
            "data": data_yaml,
            "epochs": epochs,
            "imgsz": img_size,
            "batch": batch_size,
            "lr0": lr0,
            "optimizer": optimizer,
            "device": device,
            "patience": patience,
            "momentum": momentum,
            "save": True,
            "exist_ok": True,
            "pretrained": True,
            "verbose": True,
            "project": f"yolo{self.model_version}_finetuning",
            "name": name,
            "resume": resume,
            # Disable default data augmentation for controlled experiments
            "hsv_h": 0.0,  # Hue augmentation
            "hsv_s": 0.0,  # Saturation augmentation
            "hsv_v": 0.0,  # Value augmentation
            "degrees": 0.0,  # Rotation
            "translate": 0.0,  # Translation
            "scale": 0.0,  # Scaling
            "shear": 0.0,  # Shear
            "perspective": 0.0,  # Perspective transform
            "flipud": 0.0,  # Vertical flip
            "fliplr": 0.0,  # Horizontal flip
            "mosaic": 0.0,  # Mosaic augmentation
            "mixup": 0.0,  # Mixup augmentation
            "copy_paste": 0.0,  # Copy-paste augmentation
            **kwargs,
        }

        print("\n" + "=" * 60)
        print(f"Starting training: YOLO{self.model_version}{self.model_size}")
        print("=" * 60)

        train_results = self.model.train(**train_params)

        # Get best model path from training results
        best_path = train_results.save_dir / "weights" / "best.pt"

        print(f"\n✓ Training complete! Best model saved to: {best_path}")

        return {"best_path": str(best_path), "train_results": train_results}

    def test(self, data_yaml: str, best_path: str, conf=0.25, iou=0.45, **kwargs):
        """
        Test the best model on test set.

        Args:
            data_yaml: Path to YAML configuration file
            best_path: Path to best.pt from training
            conf: Confidence threshold
            iou: IoU threshold for NMS
            **kwargs: Additional validation arguments

        Returns:
            dict: Test metrics (precision, recall, mAP50, mAP50-95)
        """
        # Load best model
        self.model = YOLO(best_path)
        print(f"✓ Loaded best model: {best_path}")

        # Run test evaluation
        test_results = self.model.val(
            data=data_yaml, split="test", conf=conf, iou=iou, **kwargs
        )

        # Display results
        print("\n" + "=" * 60)
        print("TEST METRICS")
        print("=" * 60)
        print(f"mAP50        : {test_results.box.map50:.4f}")
        print(f"mAP50-95     : {test_results.box.map:.4f}")
        print(f"Precision    : {test_results.box.mp:.4f}")
        print(f"Recall       : {test_results.box.mr:.4f}")
        print("=" * 60)

        metrics = {
            "precision": float(test_results.box.mp),
            "recall": float(test_results.box.mr),
            "mAP50": float(test_results.box.map50),
            "mAP50_95": float(test_results.box.map),
        }

        return {"metrics": metrics}

## 7. Apply Custom Data Augmentation
Inject custom dataset class into YOLO training pipeline (if enabled).

## 8. Train and Test YOLO
Execute the complete training and testing workflow.

In [ ]:
if data_augmentation.value:
    print("✓ Custom data augmentation enabled")

    from ultralytics.data.build import build_yolo_dataset

    # Save reference to original builder
    original_build = build_yolo_dataset

    def custom_build_yolo_dataset(
        cfg,
        img_path,
        batch,
        data,
        mode="train",
        rect=False,
        stride=32,
        multi_modal=False,
    ):
        """
        Custom dataset builder that injects filter-based augmentation for training.
        """
        dataset = original_build(
            cfg, img_path, batch, data, mode, rect, stride, multi_modal
        )

        # Replace dataset class with augmented version for training only
        if mode == "train":
            dataset.__class__ = YOLODatasetCustomDA

        return dataset

    # Monkey-patch Ultralytics to use custom dataset builder
    import ultralytics.data.build
    import ultralytics.models.yolo.detect.train

    ultralytics.data.build.build_yolo_dataset = custom_build_yolo_dataset
    ultralytics.models.yolo.detect.train.build_yolo_dataset = custom_build_yolo_dataset

    print("✓ Custom dataset builder injected into training pipeline")
else:
    print("○ Custom data augmentation disabled (default YOLO augmentation)")

In [ ]:
# Initialize trainer
trainer = YOLOTrainer(model_size="u", model_version="v3")

# Train model
print("Starting training phase...")
results = trainer.train(
    data_yaml=str(config_path),
    epochs=100,
    batch_size=16,
    device="0",  # Use GPU 0, or "cpu" for CPU training
)

print(f"\nTraining results: {results}")

# Test on test set with best model
print("\n\nStarting testing phase...")
test_results = trainer.test(
    data_yaml=str(config_path), best_path=results["best_path"], device="0"
)

print(f"\nTest metrics: {test_results['metrics']}")